Imports

In [1]:
from torchvision import datasets, transforms
import torch.utils.data
import torch
import sys
import argparse
import matplotlib.pyplot as plt
from utils import * 
from utils import *
import open3d as o3d
from models import *
from collections import OrderedDict
import os, shutil, gc
from tqdm import tqdm_notebook

In [2]:
%matplotlib notebook

Args

In [3]:
parser = argparse.ArgumentParser(description='VAE training of LiDAR')
parser.add_argument('--batch_size',         type=int,   default=32,             help='size of minibatch used during training')
parser.add_argument('--use_selu',           type=int,   default=0,              help='replaces batch_norm + act with SELU')
parser.add_argument('--base_dir',           type=str,   default='runs/test',    help='root of experiment directory')
parser.add_argument('--no_polar',           type=int,   default=0,              help='if True, the representation used is (X,Y,Z), instead of (D, Z), where D=sqrt(X^2+Y^2)')
parser.add_argument('--lr',                 type=float, default=1e-3,           help='learning rate value')
parser.add_argument('--z_dim',              type=int,   default=1024,            help='size of the bottleneck dimension in the VAE, or the latent noise size in GAN')
parser.add_argument('--autoencoder',        type=int,   default=1,              help='if True, we do not enforce the KL regularization cost in the VAE')
parser.add_argument('--atlas_baseline',     type=int,   default=0,              help='If true, Atlas model used. Also determines the number of primitives used in the model')
parser.add_argument('--panos_baseline',     type=int,   default=0,              help='If True, Model by Panos Achlioptas used')
parser.add_argument('--kl_warmup_epochs',   type=int,   default=150,            help='number of epochs before fully enforcing the KL loss')
parser.add_argument('--debug', action='store_true')

_StoreTrueAction(option_strings=['--debug'], dest='debug', nargs=0, const=True, default=False, type=None, choices=None, help=None, metavar=None)

In [4]:
args = parser.parse_args([])
args

Namespace(atlas_baseline=0, autoencoder=1, base_dir='runs/test', batch_size=32, debug=False, kl_warmup_epochs=150, lr=0.001, no_polar=0, panos_baseline=0, use_selu=0, z_dim=1024)

In [5]:
# model_file = '/home/sabyasachi/Projects/ati/ati_motors/adversarial_based/prashVAE/new_runs/unet_more_layers_restarted_with_more_data_ctd/models/gen_118.pth'
model_file = '/home/saby/Projects/ati/ati_motors/adversarial_based/static_reconstruction_method/second_attempt_triple_data_restarted_correctly_1024/models/gen_150.pth'

MODEL_USED_DATA_PARALLEL = False

Load Model

In [6]:
model = VAE(args, n_filters=64).cuda()
print("Loading model from {}".format(model_file))
network=torch.load(model_file)

if MODEL_USED_DATA_PARALLEL:
    # original saved file with DataParallel
    state_dict = network
    # create new OrderedDict that does not contain `module.`
    new_state_dict = OrderedDict()

    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v

    # load params
    model.load_state_dict(new_state_dict)
else:
    model.load_state_dict(network)
model.eval()

Loading model from /home/saby/Projects/ati/ati_motors/adversarial_based/static_reconstruction_method/second_attempt_triple_data_restarted_correctly_1024/models/gen_150.pth


VAE(
  (encoder_conv2d_a): Sequential(
    (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (encoder_leakyrelu_b): Sequential(
    (0): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (encoder_conv2d_A): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (encoder_leakyrelu_B): Sequential(
    (0): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (encoder_conv2d_1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  )
  (encoder_leakyrelu_2): Sequential(
    (0): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (encoder_conv2d_c): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (encoder_leakyrelu_d): Sequential(
    (0): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (encoder_conv2d_C): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

Load data

In [7]:
def getint(name):
    return int(name.split('.')[0])
    
def draw_pcd(pcd, where='opn_nb'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()
        
def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()

In [8]:
FILTER_THRESHOLD = 0.1

In [9]:
base_folder_path = "/home/saby/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/testing"

pair_folder = "24"

range_npy_folder = "_out_out_npy"
output_pcd_folder = "unet_pcd"  + "_manual_filtered_" + str(FILTER_THRESHOLD)

test_folder_path = os.path.join(base_folder_path, pair_folder, range_npy_folder)
out_folder_path = os.path.join(base_folder_path, pair_folder, output_pcd_folder)

In [10]:
# test_folder = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/validate_idea/pair_24_transform/dynamic_out_npy"
# out_folder = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/validate_idea/pair_24_transform/second_attempt_triple_data_1024_model_out_pcd" + "_filtered_" + str(FILTER_THRESHOLD)

# test_folder = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/few_dynamic_runs/110k/dynamic/scarce_1/_out_out_npy"
# out_folder = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/few_dynamic_runs/110k/dynamic/scarce_1/_model_out"
test_files  = sorted(os.listdir(test_folder_path), key=getint)

In [11]:
out_folder_path

'/home/saby/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/testing/24/unet_pcd_manual_filtered_0.1'

In [12]:
if not os.path.exists(out_folder_path):
    os.makedirs(out_folder_path)
else:
    shutil.rmtree(out_folder_path)
    os.makedirs(out_folder_path)

Evaluate on data

In [13]:
process_input = from_polar if args.no_polar else lambda x : x

In [14]:
LIDAR_RANGE = 100

In [15]:
ply_idx = 1
for test_file in test_files:
    # Load corresponding dataset batch
    print("processing {}".format(test_file))
    dataset_val = np.load(os.path.join(test_folder_path, test_file))
    dataset_val = preprocess(dataset_val, LIDAR_RANGE)
    dataset_val = dataset_val.astype('float32')
    val_loader  = torch.utils.data.DataLoader(dataset_val, batch_size=args.batch_size,
                        shuffle=False, num_workers=12, drop_last=False)

    print("done")
    recons=[]
    recon_flag = 0.0
    total_flag = 0.0
    for i, img_data in tqdm_notebook(enumerate(val_loader), total=len(val_loader)):
        dynamic_img = img_data.cuda()

        recon = model(process_input(dynamic_img))

        recons=recon
        recons_temp=np.array(recons.detach().cpu())
        
        for frame_num in range(recons_temp.shape[0]):
            frame = from_polar(recons[frame_num:frame_num+1,:,:,:]).detach().cpu().numpy()[0]
            dynamic_frame = from_polar(dynamic_img[frame_num:frame_num+1,:,:,:]).detach().cpu().numpy()[0]
            
            frame = frame * LIDAR_RANGE
            dynamic_frame = dynamic_frame * LIDAR_RANGE
            
            filtered_arr = np.zeros((frame.shape))
            for i in range(frame.shape[1]):
                for j in range(frame.shape[2]):
                    dist = np.linalg.norm(frame[:,i,j] - dynamic_frame[:,i,j])
                    if dist > FILTER_THRESHOLD:
                        filtered_arr[:,i,j] = frame[:,i,j]
                        recon_flag += 1
                    else:
                        filtered_arr[:,i,j] = dynamic_frame[:,i,j]
                    total_flag += 1
            
            some_pcd = o3d.geometry.PointCloud()
            some_arr = filtered_arr.reshape((3,-1)).T
            some_pcd.points = o3d.utility.Vector3dVector(some_arr)
            pcd_fname = str(ply_idx) + ".ply"
            single_pcd_path = os.path.join(out_folder_path, pcd_fname)
            o3d.io.write_point_cloud(single_pcd_path, some_pcd)
            ply_idx += 1
        gc.collect()
        
    print("Fraction of reconstructed points : {}".format(recon_flag / total_flag))

processing 0.npy
done


/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app



Fraction of reconstructed points : 0.7353896895004193
